## INSY 5336 Python Final Project


### Project Logic: 
* This project is about web scraping
* Here we are asked to scrape metacritic website for the movie, cast and genre details of the top 500 movies.
* The logic of the project involves scraping the mentioned details and saving it to the csv file.
* User can check the details such as: Cast and genre details of the user input movie
* user can check the movies and genre summary of their favourite actor/actress, by entering their name
* users can compare 2 actors/actresses based on the genres they have acted in and the similarity score of the same.

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [203]:
import csv
import pandas as pd
import requests
from requests import get
import re
from bs4 import BeautifulSoup
import time

In [247]:
#In this project i have mainly used pandas dataframe to store the data scraped from web.
#The below logic is used to scrape the metacritic to get the top 500 movie titles and store them in dataframe

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

movie_titles=[]
movie_links=[]

#each page has 100 movies, so we loop 5 times to get 500 movies
for i in range(5):
    
    url = "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?page="+str(i)
    response = get(url,headers = headers)
    
    if response.status_code == 200:
        
        #BeautifulSoup helps in getting the page content in html format
        movies_soup = BeautifulSoup(response.content, 'html.parser')
        movie_container = movies_soup.find_all('td', class_ = 'clamp-summary-wrap')
        
        #we have 100 movies in the container, loop through each to get the title
        for movie in movie_container:
            
            #h3 tag contains the movie title
            movie_title = movie.find('h3').text
            movie_titles.append(movie_title)
            
            movie_link_anchor = movie.find('a', class_ = 'title')
            
            #below line is used to build an url which can be used to access that particular movie
            movie_link = "https://www.metacritic.com"+movie_link_anchor.get('href', None)
            movie_links.append(movie_link)

print("Total number of Movie Records-->",len(movie_titles))
print("Total number of Movie Links-->",len(movie_links))

#creating the dataframe with Movie_Titles column containing all the movie titles
df = pd.DataFrame(movie_titles, columns = ['Movie_Titles'])
df.head()


Total number of Movie Records--> 500
Total number of Movie Links--> 500


,Movie_Titles
0,Citizen Kane
1,The Godfather
2,Rear Window
3,Casablanca
4,Boyhood


In [228]:
#similarly the below logic is used to fetch the details link for each movie
movie_links_detail=[]
for movie_link in movie_links:
    movie_page = requests.get(movie_link, headers = headers)
    movie_page_soup = BeautifulSoup(movie_page.content, "html.parser")
    movie_page_div = movie_page_soup('div', class_= 'see_all pad_btm1')
    for div in movie_page_div:
        detail_link = "https://www.metacritic.com"+div.find('a').get('href', None)
        movie_links_detail.append(detail_link)
        
print(movie_links_detail)

['https://www.metacritic.com/movie/citizen-kane-1941/details', 'https://www.metacritic.com/movie/the-godfather/details', 'https://www.metacritic.com/movie/rear-window/details', 'https://www.metacritic.com/movie/casablanca/details', 'https://www.metacritic.com/movie/boyhood/details', 'https://www.metacritic.com/movie/three-colors-red/details', 'https://www.metacritic.com/movie/vertigo-1958/details', 'https://www.metacritic.com/movie/notorious-1946/details', 'https://www.metacritic.com/movie/singin-in-the-rain/details', 'https://www.metacritic.com/movie/city-lights/details', 'https://www.metacritic.com/movie/moonlight-2016/details', 'https://www.metacritic.com/movie/intolerance/details', 'https://www.metacritic.com/movie/pinocchio-1940/details', 'https://www.metacritic.com/movie/touch-of-evil/details', 'https://www.metacritic.com/movie/the-lady-vanishes-1938/details', 'https://www.metacritic.com/movie/the-treasure-of-the-sierra-madre/details', 'https://www.metacritic.com/movie/pans-labyr

In [243]:

#Below logic is used to fetch the genres of each movie using the detailed link
genres=[]
count=1

for detail_url in movie_links_detail:
    #time.sleep(1)
    print("URL ",count,"\n",detail_url)
    sub_response = get(detail_url,headers = headers)

    if sub_response.status_code == 200:
        sub_movies_soup = BeautifulSoup(sub_response.content, 'html.parser')
        sub_movie_container = sub_movies_soup.find_all('tr', class_='genres')
        movie_genres_withLabel = sub_movie_container[0].text
        movie_genres=movie_genres_withLabel.split(':')[1].split(',')

        #removing spaces and \n
        for genre_index in range(0,len(movie_genres)):
            movie_genres[genre_index] = movie_genres[genre_index].strip()

        genres.append(movie_genres)
        print("movie_genres--->",movie_genres)
        print("")
    else:
        print("something went wrong, please run again!")
        break
    count+=1

        
print(genres)

URL  1 
 https://www.metacritic.com/movie/citizen-kane-1941/details
movie_genres---> ['Drama', 'Mystery']

URL  2 
 https://www.metacritic.com/movie/the-godfather/details
movie_genres---> ['Drama', 'Thriller', 'Crime']

URL  3 
 https://www.metacritic.com/movie/rear-window/details
movie_genres---> ['Mystery', 'Thriller']

URL  4 
 https://www.metacritic.com/movie/casablanca/details
movie_genres---> ['Drama', 'Romance', 'War']

URL  5 
 https://www.metacritic.com/movie/boyhood/details
movie_genres---> ['Drama']

URL  6 
 https://www.metacritic.com/movie/three-colors-red/details
movie_genres---> ['Drama', 'Mystery', 'Romance']

URL  7 
 https://www.metacritic.com/movie/vertigo-1958/details
movie_genres---> ['Mystery', 'Thriller', 'Romance']

URL  8 
 https://www.metacritic.com/movie/notorious-1946/details
movie_genres---> ['Drama', 'Thriller', 'Romance', 'Film-Noir']

URL  9 
 https://www.metacritic.com/movie/singin-in-the-rain/details
movie_genres---> ['Comedy', 'Romance', 'Musical']

U

movie_genres---> ['Documentary']

URL  73 
 https://www.metacritic.com/movie/a-separation/details
movie_genres---> ['Drama']

URL  74 
 https://www.metacritic.com/movie/wall-e/details
movie_genres---> ['Adventure', 'Sci-Fi', 'Romance', 'Animation', 'Family']

URL  75 
 https://www.metacritic.com/movie/carol/details
movie_genres---> ['Drama', 'Romance']

URL  76 
 https://www.metacritic.com/movie/before-midnight/details
movie_genres---> ['Drama', 'Romance']

URL  77 
 https://www.metacritic.com/movie/amour/details
movie_genres---> ['Drama', 'Romance']

URL  78 
 https://www.metacritic.com/movie/dunkirk/details
movie_genres---> ['Action', 'Drama', 'History', 'Thriller', 'War']

URL  79 
 https://www.metacritic.com/movie/the-man-who-shot-liberty-valance/details
movie_genres---> ['Drama', 'Western']

URL  80 
 https://www.metacritic.com/movie/the-searchers/details
movie_genres---> ['Adventure', 'Drama', 'Western']

URL  81 
 https://www.metacritic.com/movie/45-years/details
movie_genres---

movie_genres---> ['Drama']

URL  146 
 https://www.metacritic.com/movie/apocalypse-now-final-cut/details
movie_genres---> ['Drama', 'Mystery', 'War']

URL  147 
 https://www.metacritic.com/movie/chinatown/details
movie_genres---> ['Drama', 'Mystery', 'Thriller', 'Crime']

URL  148 
 https://www.metacritic.com/movie/the-florida-project/details
movie_genres---> ['Drama']

URL  149 
 https://www.metacritic.com/movie/timbuktu/details
movie_genres---> ['Drama']

URL  150 
 https://www.metacritic.com/movie/frenzy/details
movie_genres---> ['Thriller']

URL  151 
 https://www.metacritic.com/movie/brief-encounter/details
movie_genres---> ['Drama', 'Romance']

URL  152 
 https://www.metacritic.com/movie/cool-hand-luke/details
movie_genres---> ['Drama', 'Crime']

URL  153 
 https://www.metacritic.com/movie/little-women-1933/details
movie_genres---> ['Drama', 'Romance', 'Family']

URL  154 
 https://www.metacritic.com/movie/maborosi/details
movie_genres---> ['Drama']

URL  155 
 https://www.metacr

movie_genres---> ['Action', 'Drama', 'War']

URL  218 
 https://www.metacritic.com/movie/ben-hur-1959/details
movie_genres---> ['Adventure', 'Drama', 'History', 'War']

URL  219 
 https://www.metacritic.com/movie/after-hours/details
movie_genres---> ['Thriller', 'Comedy']

URL  220 
 https://www.metacritic.com/movie/the-crying-game/details
movie_genres---> ['Thriller', 'Romance', 'Crime']

URL  221 
 https://www.metacritic.com/movie/great-expectations-1947/details
movie_genres---> ['Adventure', 'Drama', 'Mystery', 'Romance']

URL  222 
 https://www.metacritic.com/movie/burning/details
movie_genres---> ['Drama', 'Mystery']

URL  223 
 https://www.metacritic.com/movie/winters-bone/details
movie_genres---> ['Drama', 'Thriller']

URL  224 
 https://www.metacritic.com/movie/hope/details
movie_genres---> ['Drama', 'Romance']

URL  225 
 https://www.metacritic.com/movie/gett-the-trial-of-viviane-amsalem/details
movie_genres---> ['Drama']

URL  226 
 https://www.metacritic.com/movie/i-called-h

movie_genres---> ['Drama', 'Crime', 'Film-Noir']

URL  292 
 https://www.metacritic.com/movie/the-hidden-fortress/details
movie_genres---> ['Adventure', 'Drama']

URL  293 
 https://www.metacritic.com/movie/still-walking/details
movie_genres---> ['Drama']

URL  294 
 https://www.metacritic.com/movie/gangs-of-wasseypur/details
movie_genres---> ['Action', 'Drama', 'Thriller', 'Crime']

URL  295 
 https://www.metacritic.com/movie/the-last-detail/details
movie_genres---> ['Drama', 'Comedy']

URL  296 
 https://www.metacritic.com/movie/when-the-levees-broke-a-requiem-in-four-acts/details
movie_genres---> ['Documentary']

URL  297 
 https://www.metacritic.com/movie/borat-cultural-learnings-of-america-for-make-benefit-glorious-nation-of-kazakhstan/details
movie_genres---> ['Comedy']

URL  298 
 https://www.metacritic.com/movie/minari/details
movie_genres---> ['Drama']

URL  299 
 https://www.metacritic.com/movie/barry-lyndon/details
movie_genres---> ['Adventure', 'Drama', 'History', 'War']

U

movie_genres---> ['Drama', 'Comedy']

URL  365 
 https://www.metacritic.com/movie/manuscripts-dont-burn/details
movie_genres---> ['Drama']

URL  366 
 https://www.metacritic.com/movie/in-transit/details
movie_genres---> ['Documentary', 'News']

URL  367 
 https://www.metacritic.com/movie/three-billboards-outside-ebbing-missouri/details
movie_genres---> ['Drama', 'Comedy', 'Crime']

URL  368 
 https://www.metacritic.com/movie/quest-2017/details
movie_genres---> ['Music', 'Family', 'Documentary']

URL  369 
 https://www.metacritic.com/movie/au-revoir-les-enfants/details
movie_genres---> ['Drama', 'War']

URL  370 
 https://www.metacritic.com/movie/a-film-unfinished/details
movie_genres---> ['Drama', 'History', 'Documentary']

URL  371 
 https://www.metacritic.com/movie/things-to-come/details
movie_genres---> ['Drama']

URL  372 
 https://www.metacritic.com/movie/the-grand-budapest-hotel/details
movie_genres---> ['Drama', 'Comedy']

URL  373 
 https://www.metacritic.com/movie/winter-sleep

movie_genres---> ['Family', 'Documentary']

URL  440 
 https://www.metacritic.com/movie/halloween-1978/details
movie_genres---> ['Thriller', 'Horror']

URL  441 
 https://www.metacritic.com/movie/eraserhead/details
movie_genres---> ['Horror']

URL  442 
 https://www.metacritic.com/movie/the-velvet-underground/details
movie_genres---> ['Music', 'Documentary']

URL  443 
 https://www.metacritic.com/movie/medium-cool/details
movie_genres---> ['Drama']

URL  444 
 https://www.metacritic.com/movie/can-you-bring-it-bill-t-jones-and-d-man-in-the-waters/details
movie_genres---> ['Documentary']

URL  445 
 https://www.metacritic.com/movie/fire-at-sea/details
movie_genres---> ['Documentary']

URL  446 
 https://www.metacritic.com/movie/the-fugitive/details
movie_genres---> ['Action', 'Drama', 'Mystery', 'Thriller', 'Crime']

URL  447 
 https://www.metacritic.com/movie/f-for-fake/details
movie_genres---> ['Documentary']

URL  448 
 https://www.metacritic.com/movie/about-elly-2009/details
movie_ge

In [248]:
print("Total number of genere records-->",len(genres))

#appending genres of all 500 movies to a column in dataframe 
df['genres'] = genres
df.tail()

Total number of genere records--> 500


,Movie_Titles,genres
495,Gregory Crewdson: Brief Encounters,"[Biography, Drama, Documentary]"
496,Moneyball,[Drama]
497,The Autobiography of Nicolae Ceausescu,[Documentary]
498,All About My Mother,[Drama]
499,The English Patient,"[Drama, Romance, War]"


In [265]:

#This logic is used to get the cast details of each movie:
#both Cast and Principal cast details are merged and the duplicates are eliminated before returning final cast list

def get_cast_details(detail_link,mv_name):
    detail_page = requests.get(detail_link, headers = headers)
    detail_soup = BeautifulSoup(detail_page.content, "html.parser")
    
    #getting the cast details, try except is used to handle the case where if the cast details are not there, 
    #we can skip and go to fetching primcipal cast details
    try:
        cast_table = detail_soup.find('table', class_ = 'credits', summary = mv_name + "'s Cast Credits")
        cast_line = cast_table.find_all('td', class_ = 'person')
    except:
        try:
            cast_table = detail_soup.find('table', class_ = 'credits', summary = mv_name + "' Cast Credits")
            cast_line = cast_table.find_all('td', class_ = 'person')
        except:
            print("looks like there are no cast details mentioned for the movie!")
            cast_line=""
    
    #getting the principal cast details, try except is used to handle the case where if the principal cast details are not there, 
    #we can skip getting it, without any error
    try:
        principal_table = detail_soup.find('table', class_ = 'credits', summary = mv_name + "'s Principal Cast Credits")
        principal_line = principal_table.find_all('td', class_ = 'person')
    except:
        try:
            principal_table = detail_soup.find('table', class_ = 'credits', summary = mv_name + "' Principal Cast Credits")
            principal_line = principal_table.find_all('td', class_ = 'person')
        except:
            print("looks like there are no principal cast details mentioned for the movie!")
            principal_line=""
    
    #appending cast and principal Cast details into separate list and merging them and returning them
    casts = [] 
    for line in cast_line:
        hyperlink = line.find('a')
        cast = hyperlink.get_text()
        casts.append(cast.strip())
        
    principal_cast=[]
    for line in principal_line:
        hyperlink = line.find('a')
        princi_cast = hyperlink.get_text()
        principal_cast.append(princi_cast.strip())
        
    return casts+principal_cast

In [372]:

#calling the get_cast_details() method to get the cast details, remove duplicates and print them for each movie 
cast_details=[]
count=0
for detail_url, movie_name in zip(movie_links_detail,movie_titles):
    print("\n",count,": ",movie_name,"--> ", detail_url, "  Movie cast:")
    movie_cast = get_cast_details(detail_url, movie_name)
    movie_cast =list(set(movie_cast))
    print(movie_cast)
    cast_details.append(movie_cast)
    count+=1


 0 :  Citizen Kane -->  https://www.metacritic.com/movie/citizen-kane-1941/details   Movie cast:
['Orson Welles', 'Everett Sloane', 'George Coulouris', 'Ruth Warrick', 'Joseph Cotten', 'Ray Collins', 'Georgia Backus', 'Erskine Sanford', 'Gus Schilling', 'Paul Stewart', 'Philip Van Zandt', 'William Alland', 'Agnes Moorehead', 'Dorothy Comingore', 'Harry Shannon', 'Fortunio Bonanova']

 1 :  The Godfather -->  https://www.metacritic.com/movie/the-godfather/details   Movie cast:
['Rudy Bond', 'Gianni Russo', 'Marlon Brando', 'John Cazale', 'Al Pacino', 'John Marley', 'Sterling Hayden', 'Robert Duvall', 'Richard Conte', 'Diane Keaton', 'Richard S. Castellano', 'Al Lettieri', 'Abe Vigoda', 'Talia Shire', 'James Caan']

 2 :  Rear Window -->  https://www.metacritic.com/movie/rear-window/details   Movie cast:
['Sara Berner', 'Ross Bagdasarian', 'Irene Winston', 'Georgine Darcy', 'Judith Evelyn', 'Jesslyn Fax', 'Raymond Burr', 'Wendell Corey', 'Grace Kelly', 'Thelma Ritter', 'Frank Cady', 'Je

['Michel Subor', 'Pierre Fabre', 'Vanna Urbino', 'Jeanne Moreau', 'Henri Serre', 'Sabine Haudepin', 'Serge Rezvani', 'Marie Dubois', 'Anny Nelsen', 'Danielle Bassiak', 'Oskar Werner', 'Bernard Largemain', 'Dominique Lacarrière', 'Elen Bober', 'Kate Noelle']

 23 :  The Wild Bunch -->  https://www.metacritic.com/movie/the-wild-bunch/details   Movie cast:
['Jorge Russek', 'L.Q. Jones', 'Ben Johnson', 'Jaime Sanchez', 'Warren Oates', "Edmond O'Brien", 'Strother Martin', 'Dub Taylor', 'William Holden', 'Emilio Fernández', 'Ernest Borgnine', 'Albert Dekker', 'Paul Harper', 'Robert Ryan', 'Bo Hopkins']

 24 :  My Left Foot -->  https://www.metacritic.com/movie/my-left-foot/details   Movie cast:
['Daniel Day-Lewis', 'Cyril Cusack', 'Declan Croghan', 'Brenda Fricker', 'Kirsten Sheridan', 'Phelim Drew', 'Ruth McCabe', 'Ray McAnally', 'Alison Whelan', 'Marie Conmee', "Hugh O'Conor", 'Pat Laffan', 'Fiona Shaw', 'Derry Power', 'Eanna MacLiam']

 25 :  The Third Man -->  https://www.metacritic.com/

['Russell Miles', 'Slim', 'Angela Burnett', 'Kaycee Moore', 'Dorothy Stengel', 'Delores Farley', 'Lawrence Pierott', 'Jack Drummond', 'Charles Bracy', 'Eugene Cherry', 'Henry G. Sanders', 'Chris Terrill', 'Homer Jai', 'Tobar Mayo', 'Johnny Smoke']

 45 :  Nashville -->  https://www.metacritic.com/movie/nashville/details   Movie cast:
['Barbara Baxley', 'Jeff Goldblum', 'Henry Gibson', 'Robert DoQui', 'Scott Glenn', 'Ronee Blakley', 'Shelley Duvall', 'Timothy Brown', 'Allen Garfield', 'Karen Black', 'Keith Carradine', 'David Arkin', 'Geraldine Chaplin', 'Ned Beatty', 'Barbara Harris']

 46 :  Ratatouille -->  https://www.metacritic.com/movie/ratatouille/details   Movie cast:
['Janeane Garofalo', 'John Ratzenberger', 'Will Arnett', 'Jake Steinfeld', "Peter O'Toole", 'Julius Callahan', 'Teddy Newton', 'Brian Dennehy', 'Peter Sohn', 'Lou Romano', 'Ian Holm', 'Brad Garrett', 'Patton Oswalt', 'Tony Fucile', 'James Remar']

 47 :  Parasite -->  https://www.metacritic.com/movie/parasite/detail

['Pinto Colvig', 'Lucille La Verne', 'Roy Atwell', 'Stuart Buchanan', 'Eddie Collins', 'Purv Pullen', 'Adriana Caselotti', 'Scotty Mattraw', 'Harry Stockwell', 'Billy Gilbert', 'Moroni Olsen', 'Marion Darlington', 'James MacDonald', 'Otis Harlan']

 68 :  The Hurt Locker -->  https://www.metacritic.com/movie/the-hurt-locker/details   Movie cast:
['Feisal Sadoun', 'Anthony Mackie', 'Jeremy Renner', 'Christopher Sayegh', 'Evangeline Lilly', 'Suhail Aldabbach', 'Sam Spruell', 'Sam Redford', 'Barrie Rice', 'Ralph Fiennes', 'Brian Geraghty', 'Nabil Koni', 'Christian Camargo', 'Guy Pearce', 'David Morse']

 69 :  Anatomy of a Murder -->  https://www.metacritic.com/movie/anatomy-of-a-murder/details   Movie cast:
['Lee Remick', 'John Qualen', 'Russ Brown', 'Eve Arden', 'Kathryn Grant', 'Brooks West', 'Murray Hamilton', 'George C. Scott', "Arthur O'Connell", 'Alexander Campbell', 'Ben Gazzara', 'Howard McNear', 'Orson Bean', 'James Stewart', 'Ken Lynch']

 70 :  Zero Dark Thirty -->  https://ww

['Joe Pesci', 'Paul Herman', 'Paul Ben-Victor', 'Al Pacino', 'Sebastian Maniscalco', 'Dascha Polanco', 'Joseph Russo', 'Ray Romano', 'Stephanie Kurtzuba', 'Robert De Niro', 'Kathrine Narducci', 'Harvey Keitel', 'Stephen Graham', 'Gary Basaraba', 'Steve Van Zandt', 'Anna Paquin', 'Domenick Lombardozzi', 'Jack Huston', 'Aleksa Palladino', 'Bobby Cannavale', 'Jesse Plemons', 'Sharon Pfeiffer', 'Jake Hoffman', 'India Ennenga', 'Aly Mang']

 87 :  Amazing Grace -->  https://www.metacritic.com/movie/amazing-grace-1972/details   Movie cast:
looks like there are no cast details mentioned for the movie!
['Aretha Franklin', 'Bernard Purdie', 'Bernard Pretty Purdie', 'Alexander Hamilton', 'Sydney Pollack', 'Reverand James Cleveland', 'Charlie Watts', 'Mick Jagger', 'C.L. Franklin', 'Chuck Rainey', 'Clara Ward']

 88 :  Grave of the Fireflies (1988) -->  https://www.metacritic.com/movie/grave-of-the-fireflies-1988/details   Movie cast:
['Tsutomu Tatsumi', 'Shelley Calene-Black', 'Akemi Yamaguchi',

...

['Thea Stabell', 'Tumi Løvik Jakobson', 'Maria Grazia Di Meo', 'Renate Reinsve', 'Deniz Kaya', 'Silje Storstein', 'Herbert Nordrum', 'Sofia Schandy Bloch', 'Marianne Krogh', 'Anna Dworak', 'Anders Danielsen Lie', 'Vidar Sandem', 'Helene Bjørnebye', 'Hans Olav Brenner', 'Savannah Schei']

 404 :  Toy Story 2 -->  https://www.metacritic.com/movie/toy-story-2/details   Movie cast:
['Tim Allen', 'John Ratzenberger', 'Laurie Metcalf', 'Estelle Harris', 'Joan Cusack', 'Jim Varney', 'Wayne Knight', 'Annie Potts', 'John Morris', 'Don Rickles', 'Joe Ranft', 'Tom Hanks', 'Kelsey Grammer', 'Jodi Benson', 'Wallace Shawn']

 405 :  Black Panther -->  https://www.metacritic.com/movie/black-panther/details   Movie cast:
['Jeremy Sample', 'Sydelle Noel', 'Andy Serkis', 'Martin Freeman', 'Sebastian Stan', 'Sterling K. Brown', 'David S. Lee', 'Florence Kasumba', 'Winston Duke', 'Angela Bassett', 'Isaach De Bankolé', 'Letitia Wright', "Lupita Nyong'o", 'Cecil M. Henry', 'Michael David Yuhl', 'John Kani',

looks like there are no principal cast details mentioned for the movie!
['Randal Douc']

 427 :  I Travel Because I Have to, I Come Back Because I Love You -->  https://www.metacritic.com/movie/i-travel-because-i-have-to-i-come-back-because-i-love-you/details   Movie cast:
looks like there are no principal cast details mentioned for the movie!
['Irandhir Santos']

 428 :  The Invisible Man -->  https://www.metacritic.com/movie/the-invisible-man-1933/details   Movie cast:
['Donald Stuart', 'Ted Billings', 'Merle Tottenham', 'Harry Stubbs', 'Dudley Digges', 'Gloria Stuart', 'William Harrigan', 'Robert Adair', 'Claude Rains', 'Henry Travers', 'Walter Brennan', 'Forrester Harvey', "Una O'Connor", 'Holmes Herbert', 'E.E. Clive']

 429 :  Scarface -->  https://www.metacritic.com/movie/scarface-1932/details   Movie cast:
['Ann Dvorak', 'Osgood Perkins', 'Edwin Maxwell', 'Henry Armetta', 'Purnell Pratt', 'C. Henry Gordon', 'Boris Karloff', 'Gus Arnheim', 'Tully Marshall', 'George Raft', 'Paul 

['Ricardo Gonzalez', 'Ernesto Lucero', 'Christopher Gabriel Núñez', 'Donal Brophy', 'Eduardo Espinosa', 'Cristofer Huitzil', 'Fernando Cardona', 'Alex Halpern', 'Gilberto Arenas', 'Gilberto Jimenez', 'Delfino Solis', 'Alfonso Velazquez', 'Abel Perez', 'Alejandro Huitzil', 'Genoel Ramírez']

 453 :  Jane -->  https://www.metacritic.com/movie/jane/details   Movie cast:
looks like there are no cast details mentioned for the movie!
['Jane Goodall']

 454 :  Jaws -->  https://www.metacritic.com/movie/jaws/details   Movie cast:
['Jeffrey Voorhees', 'Lorraine Gary', 'Carl Gottlieb', 'Roy Scheider', 'Chris Rebello', 'Jay Mello', 'Robert Shaw', 'Lee Fierro', 'Murray Hamilton', 'Jonathan Filley', 'Richard Dreyfuss', 'Craig Kingsbury', 'Jeffrey Kramer', 'Ted Grossman', 'Susan Backlinie']

 455 :  We Are the Best! -->  https://www.metacritic.com/movie/we-are-the-best!/details   Movie cast:
['Peter Eriksson', 'Ann-Sofie Rase', 'Vanja Engstr', 'Lily Moodysson', 'Mira Barkhammar', 'Lena Carlsson', 'C

['Barry Goldwater', 'Curtis LeMay', 'Errol Morris', 'John F. Kennedy', 'Richard Nixon', 'Woodrow Wilson', 'Harry Reasoner', 'Franklin Delano Roosevelt', 'Robert McNamara', 'Fidel Castro', 'Nikita Khrushchev', 'Lyndon Johnson']

 475 :  Uncle Boonmee Who Can Recall His Past Lives -->  https://www.metacritic.com/movie/uncle-boonmee-who-can-recall-his-past-lives/details   Movie cast:
looks like there are no principal cast details mentioned for the movie!
['Vien Pimdee', 'Jenjira Pongpas', 'Thanapat Saisaymar', 'Matthieu Ly', 'Sakda Kaewbuadee']

 476 :  Brokeback Mountain -->  https://www.metacritic.com/movie/brokeback-mountain/details   Movie cast:
['Randy Quaid', 'Michelle Williams', 'Valerie Planche', 'Heath Ledger', 'Kate Mara', 'Lachlan Mackintosh', 'Jake Gyllenhaal', 'Anna Faris', 'David Trimble', 'Linda Cardellini', 'Víctor Reyes', 'Anne Hathaway', 'Larry Reese', 'David Harbour', 'Marty Antonini']

 477 :  Cemetery of Splendor -->  https://www.metacritic.com/movie/cemetery-of-splen

looks like there are no principal cast details mentioned for the movie!
['Wladyslaw Gomulka', 'Edward Gierek', 'Mikhail Gorbachev', 'Prince Philip', 'Queen Elizabeth II', 'Erich Honecker', 'Mao Zedong', 'Charles de Gaulle', 'Jimmy Carter', 'Richard Nixon', 'Elena Ceausescu', 'Nicolae Ceausescu', 'Leonid Brezhnev', 'Wojciech Jaruzelski', 'Kim Ir-Sen']

 498 :  All About My Mother -->  https://www.metacritic.com/movie/all-about-my-mother/details   Movie cast:
['Fernando Fernán Gómez', 'Eloy Azorín', 'Juan José Otegui', 'Marisa Paredes', 'Carmen Balagué', 'Fernando Guillén', 'Carlos Lozano', 'José Luis Torrijo', 'Manuel Morón', 'Cecilia Roth', 'Toni Cantó', 'Candela Peña', 'Rosa Maria Sardà', 'Penélope Cruz', 'Antonia San Juan']

 499 :  The English Patient -->  https://www.metacritic.com/movie/the-english-patient/details   Movie cast:
['Clive Merrison', 'Geordie Johnson', 'Naveen Andrews', 'Julian Wadham', 'Kevin Whately', 'Torri Higginson', 'Willem Dafoe', 'Kristin Scott Thomas', 'Peter

In [373]:
print("\n Total number of cast records-->", len(cast_details))

#append Cast details to a column called "Cast" in the dataframe, final dataframe looks as shown below
df['Cast']= cast_details
df.tail()


 Total number of cast records--> 500


,Movie_Titles,genres,Cast
495,Gregory Crewdson: Brief Encounters,"[Biography, Drama, Documentary]","[Gregory Crewdson, Richard 'Rico' Sands]"
496,Moneyball,[Drama],"[Keith Middlebrook, Sergio Garcia, Chris Pratt..."
497,The Autobiography of Nicolae Ceausescu,[Documentary],"[Wladyslaw Gomulka, Edward Gierek, Mikhail Gor..."
498,All About My Mother,[Drama],"[Fernando Fernán Gómez, Eloy Azorín, Juan José..."
499,The English Patient,"[Drama, Romance, War]","[Clive Merrison, Geordie Johnson, Naveen Andre..."


In [375]:
#saving the dataframe to the csv file
df.to_csv('amrutha_movies.csv')

In [277]:

#-------------Main Program Starts Here----------------------------------

In [406]:
# Making the dataframe case insensitive, by converting the strings in the dataframe to lower case.
df1 = df
for index, row in df1.iterrows():
    
    row['Movie_Titles'] = row['Movie_Titles'].lower()
    
    gen=[]
    for string in row['genres']:
        gen.append(string.lower())
    row['genres'] = gen
    
    
    cas=[]
    for string in row['Cast']:
        cas.append(string.lower())
    row['Cast'] = cas

df1.head()

,Movie_Titles,genres,Cast
0,citizen kane,"[drama, mystery]","[orson welles, everett sloane, george coulouri..."
1,the godfather,"[drama, thriller, crime]","[rudy bond, gianni russo, marlon brando, john ..."
2,rear window,"[mystery, thriller]","[sara berner, ross bagdasarian, irene winston,..."
3,casablanca,"[drama, romance, war]","[john qualen, s.z. sakall, dooley wilson, enri..."
4,boyhood,[drama],"[andrea chen, sharee fowler, zoe graham, alyss..."


In [432]:

#when user enters a person as an option, then fetch the movies acted by him and the genres of those movies
def get_genre_cast_of_person(person):
    person = person.strip().lower()
            
    #dataframe.apply is used to filter the records where the Cast contains the input person
    mask = df.Cast.apply(lambda a: person in a)
    df2 = df[mask]
    
    #this returns the final string of comma separated movies acted by that person
    his_movies = ','.join(list(df2['Movie_Titles']))
    
    #loop through genres of his movies and keep the count for his each genre in the dictionary
    his_genres={}
    for index, row in df2.iterrows():
        for string in row['genres']:
            his_genres[string] = his_genres.get(string,0)+1
    
    #if his genres and his movies are empty that means that, we have an input person name wrong, 
    #so raise an exception, otherwise return his_movies and his_genres 
    if not his_movies and not his_genres:
        raise Exception("Invalid input")
    else:
        return his_movies,his_genres
    

In [447]:

#This method is used to compute the similarity score for 2 people based on their genres
import math
def compute_similarity_score(genre_person1, genre_person2):
    
    similarity_score=0
    
    combined_list = list(set(list(genre_person1)+list(genre_person2)))
    genre_person1_new={}
    genre_person2_new={}
    
    #loop through combined genres (duplicates are removed) of both persons and keep the count in dictionary
    for string in combined_list:
        genre_person1_new[string] = genre_person1.get(string,0)
        genre_person2_new[string] = genre_person2.get(string,0)
    
    #convert the values of genre dictionary for each person into list to produce genre vectors
    vector1=list(genre_person1_new.values())
    vector2=list(genre_person2_new.values())
    
    #loop through genre vectors, take the difference and square them and sum them to get the sum of differences
    for i in range(len(vector1)):
        similarity_score += (vector1[i] - vector2[i])**2
       
    #take the square root of sum of differences to get the similarity score.
    similarity_score = round(math.sqrt(similarity_score), 2)
    return similarity_score
    
    

In [448]:
#User input validation, loop untill the user enters correct option
while True:
    print("*********************************************************************************************************************")
    option=input("What do you want to check on Metacrtitics? (Please choose ‘movie’, ‘people’, or ‘comparison’,'quit'):\n")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
    option = option.lower()
    if option != "movie" and option != "people" and  option != "comparison" and  option != "quit":
        print("Invalid Input option, accepted options are: ‘movie’, ‘people’, or ‘comparison’")
    else:
        
        #if the option is movie, get the cast and genre details of the given movie, 
        #try catch has been used to handle invalid input
        if option == "movie":
            input_movie = input("What movie do you want to check?\n")
            print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
            input_movie = input_movie.strip().lower()
            
            try:
                cast_and_genre = df[df['Movie_Titles'].isin([input_movie])]
                cast_output = ','.join(list(cast_and_genre['Cast'])[0])
                genre_output = ','.join(list(cast_and_genre['genres'])[0])
                
                print(f"The cast of the movie '{input_movie}' includes:\n {cast_output}")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
                print(f"The genre of the movie '{input_movie}' is {genre_output}")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

            except:
                print("Invalid Movie name, please try again by entering valid movie name")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
        
        #if the option is people, get all the movies and genres acted by him, 
        #try catch has been used to handle invalid input   
        elif option == "people":
            
            try:
                input_person = input("Who do you want to check?\n")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
                movies_of_person,genre_of_person = get_genre_cast_of_person(input_person)
                
                print(f"'{input_person}' has acted in:\n{movies_of_person}")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
                print(f"{input_person}'s most often played genres are:\n{genre_of_person}")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
            except:
                print("Invalid Person name, please try again by entering valid person name")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
            
        #if the option is comparison, get all the movies and genres acted by both people and compute the similarity score, 
        #try catch has been used to handle invalid input  
        elif option == "comparison":
            try:
                print("Who do you want to Compare?")
                input_person1 = input("person1: ")
                input_person2 = input("person2: ")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
            
                movies_of_person1,genre_of_person1 = get_genre_cast_of_person(input_person1)
                movies_of_person2,genre_of_person2 = get_genre_cast_of_person(input_person2)
            
                print(f"'{input_person1}' has acted in:\n{movies_of_person1}")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
                print(f"{input_person1}'s most often played genres are:\n{genre_of_person1}")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
                print(f"'{input_person2}' has acted in:\n{movies_of_person2}")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
                print(f"{input_person2}'s most often played genres are:\n{genre_of_person2}")
            
                similarity_score = compute_similarity_score(genre_of_person1,genre_of_person2)
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
                print(f"Based on that, they have a cosine similarity score of {similarity_score}")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
            
            except:
                print("Invalid Person name, please try again by entering valid person name")
                print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
        
        #if the input option is quit, then terminate the loop, this is the only way to exit
        else:
            print("Good Bye!")
            break
            
            

*********************************************************************************************************************
What do you want to check on Metacrtitics? (Please choose ‘movie’, ‘people’, or ‘comparison’,'quit'):
comparison
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Who do you want to Compare?
person1: Ben Affleck 
person2: Liam Neeson 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Invalid Person name, please try again by entering valid person name
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

*********************************************************************************************************************
What do you want to check on Metacrtitics? (Please choose ‘movie’, ‘people’, or ‘comparison’,'quit'):
comparison
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~